In [92]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt

In [93]:
##all

df = pd.read_csv('threeplanetsystemsinfo_strict.csv')

columns_to_keep = ['kepmag', 'rhostar', 'mstar', 'logg', '[M/H]', 'n', 'min_p', 
                   'max_p', 'mean_p', 'range_p', 'log_range_p', 'normstd_p', 'mean_dpp', 
                     'logstd_p', 'min_r','max_r', 
                   'mean_r','mean_rrat','std_r','normstd_r','subearth', 'supearth', 'subnep', 
                   'nepplus', 'N_first', 'N_second', 'res_frac','min_abs_delta', 'med_abs_delta']  # all summary stats

#columns_to_keep = ['gapc', 'gini', 'qdisp', 'meanzeta', 'min_abs_delta', 'med_abs_delta']  # select summary stats

#columns_to_keep = ['minper', 'maxper', 'meanper', 'perrange', 'lograngep', 'normstdp', 'meandpp', 
#                   'gapc', 'gini', 'qdisp', 'meanzeta', 'min_abs_delta', 'med_abs_delta']  # sans stellar and discrete

#columns_to_keep = ['gapc', 'gini', 'logstd_p', '[M/H]', 'normstd_p']  # manhandling for a good radius dispersion plot
#columns_to_keep = ['n','min_p','max_p','mean_p','logstd_p','mean_dpp','gini']

summary_stats = df[['KOI']+columns_to_keep]

gapc=df['gapc']

summary_stats.head()


,KOI,kepmag,rhostar,mstar,logg,[M/H],n,min_p,max_p,mean_p,...,normstd_r,subearth,supearth,subnep,nepplus,N_first,N_second,res_frac,min_abs_delta,med_abs_delta
0,41,11.197,0.43884,1.10,4.103,0.098,3,6.887077,35.333574,18.345519,...,0.250699,0,2,1,0,0,0,0.00,0.069569,0.075283
1,70,12.498,1.91702,0.94,4.452,0.103,5,3.696122,77.611343,23.567564,...,0.471632,2,1,2,0,0,1,0.25,0.010009,0.075051
2,85,11.018,0.58657,1.25,4.219,0.122,3,2.154910,8.131205,5.382017,...,0.282336,0,2,1,0,0,1,0.50,0.008862,0.051208
3,94,12.205,0.66796,1.18,4.243,0.064,4,3.743158,54.320059,22.707461,...,0.585788,0,1,1,2,0,0,0.00,0.071741,0.071758
4,111,12.596,1.02446,0.82,4.385,-0.409,3,11.427545,51.755360,28.950411,...,0.063087,0,0,3,0,0,0,0.00,0.035582,0.064464


In [94]:
# Split data into train and test set

X_train, X_test, gapc_train, gapc_test = train_test_split(summary_stats[columns_to_keep], gapc, test_size=0.2, random_state=42)

# Initialize Lasso regression model
lasso = Lasso(alpha=0.1)  # Adjust alpha based on your data

# Fit the model
lasso.fit(X_train, gapc_train)

Lasso(alpha=0.1)

In [95]:
# Predictions
gapc_pred = lasso.predict(X_test)

# Metrics
mse = mean_squared_error(gapc_test, gapc_pred)
r2 = r2_score(gapc_test, gapc_pred)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

coefficients = pd.DataFrame({
    'Feature': summary_stats[columns_to_keep].columns,
    'Coefficient': lasso.coef_
})
print(coefficients[coefficients['Coefficient'] != 1]) 

Mean Squared Error: 0.03958458011259033
R² Score: 0.26001366916831103
          Feature  Coefficient
0          kepmag    -0.000000
1         rhostar    -0.000000
2           mstar     0.000000
3            logg    -0.000000
4           [M/H]     0.000000
5               n    -0.000000
6           min_p    -0.000000
7           max_p     0.000000
8          mean_p    -0.001968
9         range_p     0.001347
10    log_range_p     0.000000
11      normstd_p     0.000000
12       mean_dpp     0.054919
13       logstd_p     0.000000
14          min_r    -0.000000
15          max_r    -0.000000
16         mean_r    -0.000000
17      mean_rrat     0.000000
18          std_r    -0.000000
19      normstd_r     0.000000
20       subearth     0.000000
21       supearth     0.000000
22         subnep    -0.000000
23        nepplus    -0.000000
24        N_first     0.000000
25       N_second     0.000000
26       res_frac     0.000000
27  min_abs_delta    -0.000000
28  med_abs_delta     0.000000


In [81]:
#Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV

# Define alpha values to test
param_grid = {'alpha': np.logspace(-4, 1, 50)}

# Perform grid search
grid = GridSearchCV(Lasso(), param_grid, scoring='neg_mean_squared_error', cv=5)
grid.fit(X_train, gapc_train)

print(f"Best alpha: {grid.best_params_['alpha']}")
print(f"Best score: {grid.best_score_}")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.552e-01, tolerance: 9.286e-04
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.134e-02, tolerance: 8.585e-04
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.086e-01, toleranc

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.199e-01, tolerance: 1.019e-03
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.655e-01, tolerance: 9.286e-04
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.315e-01, toleranc

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.316e-01, tolerance: 1.019e-03
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.460e-01, tolerance: 9.286e-04
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.268e-01, toleranc

Best alpha: 0.00032374575428176434
Best score: -0.006696182060137032


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.003e-01, tolerance: 1.157e-03
  model = cd_fast.enet_coordinate_descent(


In [89]:
#Radius

df = pd.read_csv('threeplanetsystemsinfo_strict.csv')

columns_to_keep = ['min_r','max_r', 'mean_r',
                   'range_r','mean_rrat','std_r','normstd_r','subearth', 'supearth', 'subnep', 
                   'nepplus']  # all summary stats

#columns_to_keep = ['gapc', 'gini', 'qdisp', 'meanzeta', 'min_abs_delta', 'med_abs_delta']  # select summary stats

#columns_to_keep = ['minper', 'maxper', 'meanper', 'perrange', 'lograngep', 'normstdp', 'meandpp', 
#                   'gapc', 'gini', 'qdisp', 'meanzeta', 'min_abs_delta', 'med_abs_delta']  # sans stellar and discrete

#columns_to_keep = ['gapc', 'gini', 'logstd_p', '[M/H]', 'normstd_p']  # manhandling for a good radius dispersion plot
#columns_to_keep = ['n','min_p','max_p','mean_p','logstd_p','mean_dpp','gini']

summary_stats = df[['KOI']+columns_to_keep]

gapc=df['gapc']

summary_stats.head()


,KOI,min_r,max_r,mean_r,range_r,mean_rrat,std_r,normstd_r,subearth,supearth,subnep,nepplus
0,41,1.270,2.269,1.689000,0.999,1.230019,0.423431,0.250699,0,2,1,0
1,70,0.807,2.815,1.740600,2.008,1.788037,0.820922,0.471632,2,1,2,0
2,85,1.460,2.619,1.872667,1.159,1.190732,0.528722,0.282336,0,2,1,0
3,94,1.596,10.242,5.450500,8.646,1.889350,3.192839,0.585788,0,1,1,2
4,111,2.265,2.630,2.423333,0.365,1.057416,0.152880,0.063087,0,0,3,0


In [90]:
# Split data into train and test set

X_train, X_test, gapc_train, gapc_test = train_test_split(summary_stats[columns_to_keep], gapc, test_size=0.2, random_state=42)

# Initialize Lasso regression model
lasso = Lasso(alpha=0.1)  # Adjust alpha based on your data

# Fit the model
lasso.fit(X_train, gapc_train)

Lasso(alpha=0.1)

In [91]:
# Predictions
gapc_pred = lasso.predict(X_test)

# Metrics
mse = mean_squared_error(gapc_test, gapc_pred)
r2 = r2_score(gapc_test, gapc_pred)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

coefficients = pd.DataFrame({
    'Feature': summary_stats[columns_to_keep].columns,
    'Coefficient': lasso.coef_
})
print(coefficients[coefficients['Coefficient'] != 1]) 

Mean Squared Error: 0.05350295816564577
R² Score: -0.0001737441455151867
      Feature  Coefficient
0       min_r         -0.0
1       max_r         -0.0
2      mean_r         -0.0
3     range_r         -0.0
4   mean_rrat          0.0
5       std_r         -0.0
6   normstd_r          0.0
7    subearth          0.0
8    supearth          0.0
9      subnep         -0.0
10    nepplus         -0.0


In [44]:
##Stellar

df = pd.read_csv('threeplanetsystemsinfo_strict.csv')

columns_to_keep = ['kepmag', 'teff','rhostar', 'mstar', 'logg', '[M/H]']


summary_stats = df[['KOI']+columns_to_keep]

gapc=df['gapc']

summary_stats.head()


,KOI,kepmag,teff,rhostar,mstar,logg,[M/H]
0,41,11.197,5857,0.43884,1.10,4.103,0.098
1,70,12.498,5507,1.91702,0.94,4.452,0.103
2,85,11.018,6213,0.58657,1.25,4.219,0.122
3,94,12.205,6171,0.66796,1.18,4.243,0.064
4,111,12.596,5763,1.02446,0.82,4.385,-0.409


In [45]:
# Split data into train and test set

X_train, X_test, gapc_train, gapc_test = train_test_split(summary_stats[columns_to_keep], gapc, test_size=0.2, random_state=42)

# Initialize Lasso regression model
lasso = Lasso(alpha=0.1)  # Adjust alpha based on your data

# Fit the model
lasso.fit(X_train, gapc_train)

Lasso(alpha=0.1)

In [46]:
# Predictions
gapc_pred = lasso.predict(X_test)

# Metrics
mse = mean_squared_error(gapc_test, gapc_pred)
r2 = r2_score(gapc_test, gapc_pred)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

coefficients = pd.DataFrame({
    'Feature': summary_stats[columns_to_keep].columns,
    'Coefficient': lasso.coef_
})
print(coefficients[coefficients['Coefficient'] != 1]) 

Mean Squared Error: 0.055659577719950976
R² Score: -0.04048916460598373
   Feature  Coefficient
0   kepmag     0.000000
1     teff     0.000045
2  rhostar    -0.000000
3    mstar     0.000000
4     logg    -0.000000
5    [M/H]     0.000000
